In [18]:
import pandas as pd
import datetime

### MSSQL Server Connection  

In [1]:
import pyodbc
def sql_connection(): 
    conn = pyodbc.connect('Driver={ODBC Driver 17 for SQL Server};'
                      'Server=DESKTOP-JJV90PD;'
                      'Database=Incubyte_DB;'
                      'Trusted_Connection=yes;')

    cursor = conn.cursor()
    cursor.fast_executemany = True
    return cursor

### Extract 

In [35]:
def Read_File(FileName):
    # drop first two col
    df=pd.read_csv(FileName,sep='|').drop(['Unnamed: 0','H'],axis=1)
    df.head()
    return df

### Transform

In [38]:
def Transform(df):
    df['Open_Date']=df['Open_Date'].map(lambda x: datetime.datetime.strptime(str(x),'%Y%m%d').date())
    df['Last_Consulted_Date']=df['Last_Consulted_Date'].map(lambda x: datetime.datetime.strptime(str(x),'%Y%m%d').date())
    df['DOB']=df['DOB'].map(lambda x: datetime.datetime.strptime(str(x),'%d%m%Y').date())
    return df

### Create Table Query - MSSQL

### Load 

In [41]:
def Load(df):
    cursor=sql_connection()
    insert_query='INSERT INTO Incubyte_DB.dbo.Incubyte_table VALUES (?,?,?,?,?,?,?,?,?,?)'
    cursor.executemany(insert_query,df.values.tolist())
    cursor.commit()
    cursor.close()

## Main Run 

In [42]:
# Extract file
df=Read_File('Incubyte_data.txt')

#transform to defined format
df=Transform(df)

#load to MSSQL table
Load(df)

In [34]:
cursor=sql_connection()
cursor.execute('SELECT * FROM Incubyte_DB.dbo.Incubyte_table')
for row in cursor:
    print(row)

('Alex', '123457', datetime.date(2010, 10, 12), datetime.date(2012, 10, 13), 'MVD  ', 'Paul                                                                                                                                                                                                                                                           ', 'SA   ', 'USA  ', datetime.date(1987, 3, 6), 'A')
('John', '123458', datetime.date(2010, 10, 12), datetime.date(2012, 10, 13), 'MVD  ', 'Paul                                                                                                                                                                                                                                                           ', 'TN   ', 'IND  ', datetime.date(1987, 3, 6), 'A')
('Mathew', '123459', datetime.date(2010, 10, 12), datetime.date(2012, 10, 13), 'MVD  ', 'Paul                                                                                                                     